<a href="https://colab.research.google.com/github/h4ck4l1/datasets/blob/main/NLP_with_RNN_and_Attention/pt_en_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys,os,warnings,logging,time
from typing import Literal
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")
if "google.colab" in sys.modules:
    %pip install -q -U "tensorflow-text==2.13.0"

import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_text as tftext
import tensorflow_text.tools.wordpiece_vocab.bert_vocab_from_dataset as bert_vocab
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
%matplotlib inline
%xmode Context
tf.get_logger().setLevel("ERROR")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 41.6 MB/s eta 0:00:00
Exception reporting mode: Context


In [2]:
# tpu_cluster = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu_cluster)
# tf.tpu.experimental.initialize_tpu_system(tpu_cluster)
# strategy = tf.distribute.TPUStrategy(tpu_cluster)
strategy = tf.distribute.OneDeviceStrategy(device="/device:GPU:0")

In [3]:
examples,metadata = tfds.load("ted_hrlr_translate/pt_to_en",with_info=True,as_supervised=True)
train_raw,valid_raw = examples['train'],examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/51785 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1193 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteTPFVDC/ted_hrlr_translate-vali…

Generating test examples...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteTPFVDC/ted_hrlr_translate-test…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [4]:
for i in train_raw.take(1):
    print(i[0].numpy().decode())
    print(i[1].numpy().decode())

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


In [5]:
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_raw.map(lambda x,y:y),
    vocab_size=8000,
    reserved_tokens=["[PAD]","[UNK]","[START]","[END]"],
    bert_tokenizer_params=dict(lower_case=True),
    learn_params={}
    )
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_raw.map(lambda x,y:x),
    vocab_size=8000,
    reserved_tokens=["[PAD]","[UNK]","[START]","[END]"],
    bert_tokenizer_params=dict(lower_case=True) ,
    learn_params={}
)

In [6]:
with open("en_pt_vocab.txt","w") as f:
    for token in en_vocab:
        print(token,file=f)

with open("pt_en_vocab.txt","w") as f:
    for token in pt_vocab:
        print(token,file=f)

In [7]:
en_tokenizer = tftext.BertTokenizer("en_pt_vocab.txt",lower_case=True)
pt_tokenizer = tftext.BertTokenizer("pt_en_vocab.txt",lower_case=True)

In [8]:
def upstream(text,lang:Literal["en","pt"]):
    batch_size = tf.shape(text)[0]
    if lang == "en":
        tokens = en_tokenizer.tokenize(text).merge_dims(-2,-1)
    else:
        tokens = pt_tokenizer.tokenize(text).merge_dims(-2,-1)
    start_tokens = tf.fill(dims=[batch_size,1],value=tf.constant(value=en_vocab.index("[START]"),dtype=tokens.dtype))
    end_tokens = tf.fill(dims=[batch_size,1],value=tf.constant(value=en_vocab.index("[END]"),dtype=tokens.dtype))
    return tf.concat([start_tokens,tokens,end_tokens],axis=-1)

In [9]:
for i in train_raw.batch(3).take(1).map(lambda x,y:y):
    tokens = upstream(i,"en")
    print(tokens[:,:4])

<tf.RaggedTensor [[2, 72, 117, 79],
 [2, 87, 90, 107],
 [2, 87, 83, 149]]>


In [10]:
def preprocess(pt,en):
    pt_tokens = upstream(pt,"pt")
    en_tokens = upstream(en,"en")
    pt_tokens = pt_tokens[:,:129]
    en_tokens = en_tokens[:,:128]
    return (en_tokens.to_tensor(),pt_tokens[:,:-1].to_tensor()),pt_tokens[:,1:].to_tensor()

In [11]:
train_ds = (
    train_raw
    .shuffle(20000)
    .batch(64)
    .map(preprocess)
    .prefetch(tf.data.AUTOTUNE)
)
valid_ds = (
    valid_raw
    .shuffle(20000)
    .batch(64)
    .map(preprocess)
    .prefetch(tf.data.AUTOTUNE)
)

In [12]:
for (en_in,pt_in),pt_out in train_ds.take(1):
    print(en_in.shape)
    print(pt_in.shape)
    print(pt_out.shape)
    print(pt_in[0,:10])
    print(pt_out[0,:10])

(64, 88)
(64, 102)
(64, 102)
tf.Tensor([   2 7076   92 3033 2191   44 1871 3318   93  300], shape=(10,), dtype=int64)
tf.Tensor([7076   92 3033 2191   44 1871 3318   93  300   16], shape=(10,), dtype=int64)


In [13]:
class PositionEmbedding(keras.layers.Layer):

    def __init__(self,casting:Literal["concat","interleave"],length:int=2048,d_model:int=512,**kwargs):


        super(PositionEmbedding,self).__init__(**kwargs)
        assert d_model%2==0,f"The depth_of_model {d_model} should be even number"
        d_model = d_model//2
        positions = np.arange(length)[:,np.newaxis]
        angles = np.arange(d_model)[np.newaxis,:]/d_model
        angles = 1/(10000**angles)
        angle_rads = positions * angles
        if casting == "concat":
            self.embed = tf.concat([tf.sin(angle_rads),tf.cos(angle_rads)],axis=-1)
        else:
            self.embed = np.zeros(shape=[length,d_model])
            self.embed[:,::2] = tf.sin(angle_rads)
            self.embed[:,1::2] = tf.cos(angle_rads)



    def call(self,inputs):

        seq_l = tf.shape(inputs)[1]
        return tf.cast(inputs,self.embed.dtype) + self.embed[tf.newaxis,:seq_l,:]

In [14]:
def get_model(
    target_vocab_size:int,
    d_model:int=512,
    length:int=2048,
    enc_heads:int=3,
    dec_heads:int=4,
    num_encoder_layers:int=4,
    num_decoder_layers:int=4,
    encoder_feed_forward_units:int=2048,
    decoder_feed_forward_units:int=2048,
    encoder_dropout_rate:float=0.1,
    decoder_dropout_rate:float=0.1
    ):

    encoder_inputs = keras.layers.Input(shape=[],dtype=tf.int64)
    decoder_inputs = keras.layers.Input(shape=[],dtype=tf.int64)
    encoder_embedding = keras.layers.Embedding(len(en_vocab),d_model,mask_zero=True)(encoder_inputs)
    decoder_embedding = keras.layers.Embedding(len(pt_vocab),d_model,mask_zero=True)(decoder_inputs)
    z_enc = PositionEmbedding("concat")(encoder_embedding)
    z_dec = PositionEmbedding("concat")(decoder_embedding)

    for _ in range(num_encoder_layers):

        '''Self Attention Part'''
        z_copy = z_enc
        self_attention = keras.layers.MultiHeadAttention(enc_heads,d_model)
        z_enc = self_attention(query=z_enc,key=z_enc,value=z_enc)
        z_enc = keras.layers.Add()([z_copy,z_enc])
        z_enc = keras.layers.LayerNormalization()(z_enc)

        '''Feed Forward Part'''
        z_copy = z_enc
        z_enc = keras.layers.Dense(encoder_feed_forward_units,"relu")(z_enc)
        z_enc = keras.layers.Dense(d_model)(z_enc)
        z_enc = keras.layers.Dropout(encoder_dropout_rate)(z_enc)
        z_enc = keras.layers.Add()([z_enc,z_copy])
        z_enc = keras.layers.LayerNormalization()(z_enc)


    for _ in range(num_decoder_layers):

        '''Masked Self Attention Part'''
        z_copy = z_dec
        masked_self_attention = keras.layers.MultiHeadAttention(dec_heads,d_model)
        z_dec = masked_self_attention(query=z_dec,key=z_dec,value=z_dec,use_causal_mask=True)
        z_dec = keras.layers.Add()([z_copy,z_dec])
        z_dec = keras.layers.LayerNormalization()(z_dec)

        '''Cross Attention Part'''
        z_copy = z_dec
        cross_attention = keras.layers.MultiHeadAttention(dec_heads,d_model)
        z_dec = cross_attention(query=z_dec,key=z_enc,value=z_enc)
        z_dec = keras.layers.Add()([z_copy,z_dec])
        z_dec = keras.layers.LayerNormalization()(z_dec)

        '''Feed Forward Part'''
        z_copy = z_dec
        z_dec = keras.layers.Dense(decoder_feed_forward_units,"relu")(z_dec)
        z_dec = keras.layers.Dense(d_model)(z_dec)
        z_dec = keras.layers.Dropout(decoder_dropout_rate)(z_dec)
        z_dec = keras.layers.Add()([z_copy,z_dec])
        z_dec = keras.layers.LayerNormalization()(z_dec)

    out = keras.layers.Dense(len(pt_vocab))(z_dec)

    return keras.Model(inputs=[encoder_inputs,decoder_inputs],outputs=[out])

In [15]:
def custom_loss(y_true,y_pred):

    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction=keras.losses.Reduction.NONE)
    loss = loss_fn(y_true,y_pred)
    mask = tf.cast(y_true != 0,loss.dtype)
    loss *= mask
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)


def custom_accuracy(y_true,y_pred):

    y_pred = tf.cast(tf.argmax(y_pred,axis=-1),y_true.dtype)
    mask = tf.cast(y_true != 0,tf.int32)
    acc = tf.cast(y_true == y_pred,tf.int32)
    acc = acc & mask
    return tf.reduce_sum(acc)/tf.reduce_sum(mask)

In [16]:
class CustomLR(keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self,d_model:int=512,warmup:int=4000,**kwargs):

        self.factor = tf.math.rsqrt(tf.cast(d_model,tf.float32)) # d_model^-0.5
        self.warmup_factor = tf.math.pow(tf.cast(warmup,tf.float32),tf.cast(-1.5,tf.float32))

    def __call__(self,step):
        step = tf.cast(step,tf.float32)
        return self.factor * tf.math.minimum(tf.math.rsqrt(step),step*self.warmup_factor)

In [17]:
with strategy.scope():
    model = get_model(
        len(pt_vocab),

        )
    model.compile(
        loss=custom_loss,
        metrics=[custom_accuracy,custom_loss],
        optimizer="adam",
        jit_compile=True,
        steps_per_execution=24
    )